In [114]:
import osmnx as ox
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import LineString
from pyproj import CRS, Transformer
from rasterio.windows import Window
from geopy.distance import geodesic

# Get the network
G = ox.graph_from_place('Isle of Wight', network_type='all')
edges = ox.graph_to_gdfs(G, nodes=False)

# Filter the major roads
major_roads = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
               'tertiary_link', 'trunk', 'trunk_link', 'residential', 'cycleway',
               'living_street', 'unclassified', 'motorway', 'motorway_link',
               'pedestrian', 'steps', 'track']
edges = edges[edges['highway'].isin(major_roads)]

# Load the DEM
dem = rasterio.open('IsleOfWightNASA_clip.tif')

# Define a Transformer from EPSG:4326 to a local projection (in this case, the British National Grid, EPSG:27700)
transformer = Transformer.from_crs(CRS('EPSG:4326'), CRS('EPSG:27700'), always_xy=True)

# Calculate the slope
def calc_slope(row):
    coords = row['geometry'].coords.xy
    start = (coords[0][0], coords[1][0])
    end = (coords[0][-1], coords[1][-1])

    start_elevation = list(dem.sample([start]))[0][0]
    end_elevation = list(dem.sample([end]))[0][0]
    dz = end_elevation - start_elevation

    if abs(dz) > 100:  # Adjust this value based on what you consider to be an "outlier"
        return 0

    start_x, start_y = transformer.transform(*start)
    end_x, end_y = transformer.transform(*end)
    dx = end_x - start_x
    dy = end_y - start_y
    distance = np.sqrt(dx ** 2 + dy ** 2)
    
    slope = (dz / distance) * 100 if distance != 0 else 0
    return slope



# Apply function to each row in the GeoDataFrame
edges['slope'] = edges.apply(calc_slope, axis=1)

# Replace NaN slope values with 0
edges['slope'] = edges['slope'].fillna(0)


# Classify slopes
edges['slope_class'] = pd.cut(edges['slope'],
                              bins=[-30, 3, 5, 8, 10, 20, np.inf],
                              labels=["0-3: flat", "3-5: mild", "5-8: medium",
                                      "8-10: hard", "10-20: extreme", ">20: impossible"],
                              right=False)
edges['slope_class'] = edges['slope_class'].fillna("0-3: flat")



# Calculate the proportion of each slope class
slope_class_distribution = round(edges['slope_class'].value_counts(normalize=True) * 100, 1)
print(slope_class_distribution)



0-3: flat          78.1
3-5: mild           8.2
5-8: medium         7.0
10-20: extreme      3.2
8-10: hard          2.3
>20: impossible     1.2
Name: slope_class, dtype: float64


In [116]:
# Create a colormap for slope classes
color_palette = ["#267300", "#70A800", "#FFAA00", "#E60000", "#A80000", "#730000"]
slope_classes = ["0-3: flat", "3-5: mild", "5-8: medium", "8-10: hard", "10-20: extreme", ">20: impossible"]
colors = dict(zip(slope_classes, color_palette))

# Calculate the mean of latitudes and longitudes
mean_latitude = edges.geometry.apply(lambda geom: geom.centroid.y).mean()
mean_longitude = edges.geometry.apply(lambda geom: geom.centroid.x).mean()

# Create a folium map centered on the mean of latitudes and longitudes
map_osm = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=11)

# Add slope information to the map
for _, row in edges.iterrows():
    color = colors.get(str(row['slope_class']), "#000000")  # default color is black
    folium.GeoJson(
        row['geometry'], 
        style_function=lambda feature, color=color: {'color': color}  # use default argument to capture color
    ).add_to(map_osm)


# Display the map 
map_osm

# Save the map
map_osm.save('map_wight_island.html')

